In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

True

In [2]:
groq_api_key=os.getenv('GROQ_API_KEY')
model = ChatGroq(model= 'Gemma2-9b-It', groq_api_key= groq_api_key)
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FA59805090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FA59806500>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import AIMessage, HumanMessage

model.invoke([HumanMessage(content= 'Hi, My name is Dishant and I am AI Researcher')])

AIMessage(content="Hello Dishant!\n\nIt's great to meet you.  As an AI researcher myself (in a sense!), I'm always interested to connect with others in the field. \n\nWhat kind of AI research are you focused on?  I'm curious to learn more about your work.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 21, 'total_tokens': 85, 'completion_time': 0.116363636, 'prompt_time': 0.002181306, 'queue_time': 0.248908843, 'total_time': 0.118544942}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--45311acb-00f3-4ae4-9068-36da7508c4c2-0', usage_metadata={'input_tokens': 21, 'output_tokens': 64, 'total_tokens': 85})

In [4]:
model.invoke(
    [
        HumanMessage(content="Hi , My name is Dishant and I am a Chief AI Engineer"),
        AIMessage(content="Hello Dishant! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Dishant, and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm ready to listen! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 99, 'total_tokens': 143, 'completion_time': 0.08, 'prompt_time': 0.0053143, 'queue_time': 0.247865399, 'total_time': 0.0853143}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--bfd7cf8a-7c51-4114-bc3e-56e5a83a8624-0', usage_metadata={'input_tokens': 99, 'output_tokens': 44, 'total_tokens': 143})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)



In [6]:
config={"configurable":{"session_id":"chat1"}}

response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [7]:
response.content

"Hello Krish, it's nice to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days?  \n\nI'm eager to learn more about your work in the field of AI.  Do you have any particular areas of expertise or interests you'd like to discuss?\n"

In [8]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish. 😊  \n\nI remembered it from our introduction!  How can I help you today? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 108, 'total_tokens': 135, 'completion_time': 0.049090909, 'prompt_time': 0.00516, 'queue_time': 0.24627279800000001, 'total_time': 0.054250909}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--8526fc64-cacd-4e1f-bbf3-83afa99d6325-0', usage_metadata={'input_tokens': 108, 'output_tokens': 27, 'total_tokens': 135})

In [9]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)


In [10]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. 👋  \n\nWhat can I do for you today?  \n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assistant. Answer all the questions to nest of your ability."),
        MessagesPlaceholder(variable_name= 'messages')
    ]
)

chain = prompt | model

In [12]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish! It's nice to meet you. \n\nWhat can I do for you today? I'm ready for any questions you have! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 29, 'total_tokens': 65, 'completion_time': 0.065454545, 'prompt_time': 0.002118396, 'queue_time': 0.252698324, 'total_time': 0.067572941}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c83394ab-f89e-44b6-9de2-5549ad40880c-0', usage_metadata={'input_tokens': 29, 'output_tokens': 36, 'total_tokens': 65})

In [13]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [14]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)


In [15]:
response

AIMessage(content="Hello Krish, it's nice to meet you! 👋 \n\nI'm happy to help with any questions you have. Just ask away! 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 29, 'total_tokens': 65, 'completion_time': 0.065454545, 'prompt_time': 0.002128736, 'queue_time': 0.247703274, 'total_time': 0.067583281}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7c315d25-6777-48f2-96c0-7de6e9c5eed4-0', usage_metadata={'input_tokens': 29, 'output_tokens': 36, 'total_tokens': 65})

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish.  😊 \n\nI remember that you told me! \n\n\n\n'

In [17]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [18]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! 🤝\n\nमुझे बताओ, मैं कैसे तुम्हारी मदद कर सकता हूँ? 😊\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [19]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [20]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्णा! मैं आपकी मदद करने के लिए तैयार हूँ।  क्या मैं आपके लिए कुछ कर सकता हूँ?\n'

In [21]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [23]:
response.content

'आपका नाम कृष्णा है। 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [24]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\disha\miniconda3\envs\rag-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\disha\miniconda3\envs\rag-env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\disha\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, 

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [25]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences.  \n\nTo figure out what ice cream you like, you could:\n\n* **Think about your favorite flavors.** Do you like chocolate, vanilla, fruity flavors, or something more unique?\n* **Remember what you've enjoyed in the past.** Have you had any ice cream that you particularly loved?\n* **Consider what you're in the mood for.** Are you craving something sweet, creamy, or refreshing?\n\n\nLet me know if you want to brainstorm some ice cream flavors!\n"

In [26]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2" 😄  \n\n\n\n\nIs there anything else I can help you with?\n'

In [27]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [28]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you. So I don't know your name. 😊\n\nWould you like to tell me?\n"

In [29]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. So I don't know what math problem you asked. \n\nIf you'd like to ask me a math problem now, I'm happy to help! 😊  \n\n"